In [5]:
import numpy as np
import matplotlib.pyplot as plt
import os
join = os.path.join
from tqdm import tqdm
from skimage import transform 
from torch.utils.data import Dataset, DataLoader
import torch.multiprocessing as mp
import sys
sys.path.append('../../modified_medsam_repo')
from segment_anything import sam_model_registry, build_sam_vit_b_multiclass
import random
from datetime import datetime
import pandas as pd
import nibabel as nib
import pickle   
from glob import glob

In [24]:
dfo = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv')

'/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/pretrained_image_encoded_slices/100206/6.npy'
'/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/segmentation_slices/100206/seg_0.npy'

splits = pd.read_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/train_val_test_split.pickle')

collect_dict = {
    'id':[],
    'slice':[],
    'image_embedding_slice_path':[],
    'segmentation_slice_path':[],
    'image_path':[],
    'bbox_0':[],
    'bbox_1':[],
    'bbox_2':[],
}

for id in dfo['id'].unique():
    tag = ''
    if id in splits['train']: tag = 'train'
    elif id in splits['val']: tag = 'val'
    elif id in splits['test']: tag = 'test'
    else: 
        print('ERR')
        break
    
    for this_slice in range(256):
        collect_dict['id'].append(id)
        collect_dict['slice'].append(this_slice)
        
        collect_dict['image_embedding_slice_path'].append(os.path.join('/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/pretrained_image_encoded_slices', str(id), f'{this_slice}.npy'))
        collect_dict['segmentation_slice_path'].append(os.path.join('/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/segmentation_slices', str(id), f'seg_{this_slice}.npy'))
        collect_dict['image_path'].append(os.path.join('/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/dir_structure_for_yolov7', tag, 'images', f'{id}_slice{this_slice}.png'))

        collect_dict['bbox_0'].append(0)
        collect_dict['bbox_1'].append(0)
        collect_dict['bbox_2'].append(256)
        collect_dict['bbox_3'].append(256)
    
df_res = pd.DataFrame(collect_dict)

df_res.to_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv', index=False)